# Class TemporalGraph

In [1]:
from functools import lru_cache
import os
import json
import networkx as nx
class TemporalGraph:
    def __init__(self, files):
        """
        Initialize TemporalGraph with a list of JSON files representing graphs at different timestamps.
        """
        self.files = files  # List of JSON file paths

    @lru_cache(maxsize=10)  # Cache the last 10 accessed timestamps
    def load_graph_at_timestamp(self, timestamp):
        """
        Load the graph for a specific timestamp from JSON and convert it to a NetworkX graph.
        """
        with open(self.files[timestamp], 'r') as f:
            data = json.load(f)
        return self._json_to_graph(data)

    def _json_to_graph(self, data):
        """
        Convert JSON data to a NetworkX graph.
        """
        graph = nx.DiGraph() if data["directed"] else nx.Graph()
        
        # Add nodes
        
        for node_type, nodes in data["node_values"].items():
            for node in nodes:
                node_id = node[-1]  # Assuming the node ID is the last element in the list
                node_attributes = dict(zip(data["node_types"][node_type], node))
                graph.add_node(node_id, **node_attributes)  # Add the node with its attributes
            
                
        # Add edges
        all_edge_types = data["relationship_types"]

        for i in data["relationship_values"] :
            
            if i[0] in all_edge_types :
                
                attributes = {}
                for j in range(len(i)-2) :
                    key = all_edge_types[i[0]][j]
                    attributes[key] = i[j]

                graph.add_edge(i[-2],i[-1],**attributes)
            else :
                graph.add_edge(i[0],i[1])
    
        
        return graph



# Loading and Wrapper

In [2]:
import time  # Ensure this is imported properly
import tracemalloc
import functools

def time_and_memory(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # Start tracking memory and time
        tracemalloc.start()
        start_time = time.time()  # Ensure time module is used correctly
        
        try:
            # Call the actual function
            result = func(*args, **kwargs)
        finally:
            # Calculate memory and time usage
            current, peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()
            end_time = time.time()
            elapsed_time = end_time - start_time

            # Print results
            print(f"Time taken by '{func.__name__}': {elapsed_time:.2f} seconds")
            print(f"Memory used by '{func.__name__}': {current / 1024:.2f} KiB (Current), {peak / 1024:.2f} KiB (Peak)")

        return result
    return wrapper

In [3]:
import glob
import re

# Natural sorting function
def natural_sort(files):
    # Extract numeric parts from filenames for sorting
    return sorted(files, key=lambda x: int(re.search(r'timestamp_(\d+)', x).group(1)))

# Get files and sort
files = glob.glob("data/supply_chain_export_1000/timestamp_*.json")
files = natural_sort(files)

# Initialize TemporalGraph
temporal_graph = TemporalGraph(files)
temporal_graph.files

['data/supply_chain_export_1000\\timestamp_0.json',
 'data/supply_chain_export_1000\\timestamp_1.json',
 'data/supply_chain_export_1000\\timestamp_2.json',
 'data/supply_chain_export_1000\\timestamp_3.json',
 'data/supply_chain_export_1000\\timestamp_4.json',
 'data/supply_chain_export_1000\\timestamp_5.json',
 'data/supply_chain_export_1000\\timestamp_6.json',
 'data/supply_chain_export_1000\\timestamp_7.json',
 'data/supply_chain_export_1000\\timestamp_8.json',
 'data/supply_chain_export_1000\\timestamp_9.json',
 'data/supply_chain_export_1000\\timestamp_10.json',
 'data/supply_chain_export_1000\\timestamp_11.json']

# Queries 

### Supplier Reliability and Costing Analysis

In [4]:
@time_and_memory
def supplier_reliability_costing_temporal(tg, timestamp, reliability_threshold, max_transportation_cost):
    """
    Analyze supplier reliability and transportation costs at a specific timestamp in a temporal graph.

    Parameters:
        tg (TemporalGraph): The temporal graph object.
        timestamp (int): The specific timestamp to analyze.
        reliability_threshold (float): The maximum acceptable reliability threshold for suppliers.
        max_transportation_cost (float): The maximum acceptable transportation cost.

    Returns:
        list: A list of suppliers meeting the criteria, including supplier ID, reliability, and transportation cost.
    """
    # Load the graph for the specified timestamp
    graph = tg.load_graph_at_timestamp(timestamp)
    
    suppliers = []
    
    # Iterate through edges to find SupplierToWarehouse relationships
    for u, v, data in graph.edges(data=True):
        
        if data.get("relationship_type") == "SupplierToWarehouse":
            
            transportation_cost = data.get("transportation_cost", 0)
            
            # Check transportation cost and reliability
            if transportation_cost <= max_transportation_cost:
                reliability = graph.nodes[u].get("reliability", 0)
                if reliability >= reliability_threshold:
                    suppliers.append((u, reliability, transportation_cost))
    
    return suppliers


In [5]:
results = supplier_reliability_costing_temporal(temporal_graph, 1, 0.95, 50)


Time taken by 'supplier_reliability_costing_temporal': 0.34 seconds
Memory used by 'supplier_reliability_costing_temporal': 6875.73 KiB (Current), 8789.43 KiB (Peak)


In [6]:
print("Suppliers meeting the criteria:")
for supplier in results:
    print(f"Supplier ID: {supplier[0]}, Reliability: {supplier[1]}, Transportation Cost: {supplier[2]}")


Suppliers meeting the criteria:
Supplier ID: S_036, Reliability: 0.9981630855575793, Transportation Cost: 27.198544570028744
Supplier ID: S_063, Reliability: 0.9673805643755703, Transportation Cost: 16.350784095737552


### Given a Supplier ID and Warehouse ID get lead time.

In [9]:
@time_and_memory
def query_lead_time_supplier_to_warehouse(temporal_graph,timestamp, supplier_id, warehouse_id):
    G = temporal_graph.load_graph_at_timestamp(timestamp)
    if G.has_edge(supplier_id, warehouse_id):
        edge_data = G[supplier_id][warehouse_id]
        if edge_data.get("relationship_type") == "SupplierToWarehouse":
            lead_time = edge_data.get("lead_time")
            return lead_time
        else:
            return None
    else:
        return None


In [10]:
supplier_id = 'S_003'
warehouse_id = 'W_143'

lead_time = query_lead_time_supplier_to_warehouse(temporal_graph,0, supplier_id, warehouse_id)
print("Lead time:",lead_time)

Time taken by 'query_lead_time_supplier_to_warehouse': 0.61 seconds
Memory used by 'query_lead_time_supplier_to_warehouse': 6884.04 KiB (Current), 8798.94 KiB (Peak)
Lead time: None


### Parts supplied by each supplier.

In [11]:
@time_and_memory
def query_supplied_part_types_for_supplier(temporal_graph,timestamp, supplier_id):
    G = temporal_graph.load_graph_at_timestamp(timestamp)
    if supplier_id in G.nodes and G.nodes[supplier_id].get("node_type") == "Supplier":
        supplied_part_types = G.nodes[supplier_id].get("supplied_part_types")
        return supplied_part_types
    else:
        return None


In [12]:
supplier_id = 'S_003'

supplied_part_types = query_supplied_part_types_for_supplier(temporal_graph,0, supplier_id)
print("Supplied part types:",supplied_part_types)

Time taken by 'query_supplied_part_types_for_supplier': 0.00 seconds
Memory used by 'query_supplied_part_types_for_supplier': 0.18 KiB (Current), 0.18 KiB (Peak)
Supplied part types: []


### Next